# MTurk Question Creation and Analysis

### Using the tutorial as inspiration for accessing the mturk API:
https://blog.mturk.com/tutorial-a-beginners-guide-to-crowdsourcing-ml-training-data-with-python-and-mturk-d8df4bdf2977

In [18]:
# Connect to the client
import boto3
MTURK_SANDBOX = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'
mturk = boto3.client('mturk',
   aws_access_key_id = "access_key_here",
   aws_secret_access_key = "secret_key_here",
   region_name='us-east-1',
#    endpoint_url = MTURK_SANDBOX # Uncomment to check sandbox account balence
)

# Check connection
print("I have $" + mturk.get_account_balance()['AvailableBalance'] + " in my Sandbox account")

I have $50.40 in my Sandbox account


### Process the debates from the generated file and add them to the question template (an XML file containing an HTML page).

In [19]:
def debates():
    debate_dict = {}
#     filename = 'output.kialo_target'
    filename = 'all_human.txt'
    with open(filename) as f:
        debates = f.readlines()        
        for d in debates:
            prompt = d.split('<EOA>')[0]
            debate_dict[prompt] = d  # We only want one debate per unique prompt; so overwrite
            
        print('Prompts with unique debates from {}:'.format(filename), len(debate_dict))
        
        for prompt, debate  in debate_dict.items():
            yield prompt, debate
            

def preprocess_question(question, xml_template, debate_text):
    # Format the debate text properly
    arguments = debate_text.split('<EOA>')
    processed_arguments = []
    i = True
    for argument in arguments:
        
        if i:
            processed_arguments.append('<bold>Debater A:</bold>  ' + argument)
        else: 
            processed_arguments.append('<bold>Debater B:</bold>  ' + argument)
        i = not i
    
    debate = '<br>\n'.join(processed_arguments)
    debate = debate.replace('<unk>', '______')
    debate = debate.replace('<unk', '______')
    
    # Insert the debate into the question HTML
    html = question.replace('<DEBATE_TEXT>', debate)
    
    with open('example.html', 'w') as f:
        f.write(html)
    
    # Insert the html into the mturk XML format
    xml_question = xml_template.replace('<QUESTION_HTML>', html)
    
#     print(xml_question)
    return xml_question

# Define and preprocess question
question_html_template = open('question.html', mode='r').read()
mturk_xml_template = open('questions.xml', mode='r').read()

i = 0
for prompt, debate in debates():
    question = preprocess_question(question_html_template, mturk_xml_template, debate)
    with open('hits/human_{}.xml'.format(i), 'w') as f:
        f.write(question)
    i += 1

Prompts with unique debates from all_human.txt: 338


In [ ]:
debate

### Create the HITs with the Amazon API

In [ ]:
import os

hits_dir = 'hits/human'

def define_hit(question):
    new_hit = mturk.create_hit(
        Title='Rate the quality of this debate.',
        Description='Read the short debate from both perspectives and rank it\'s quality.',
        Keywords='text, reading, quick, rate',
        Reward='0.10', 
        MaxAssignments=5,
        LifetimeInSeconds = 172800,
        AssignmentDurationInSeconds=600,
        AutoApprovalDelayInSeconds = 14400,
        Question=question,
    )

    print(new_hit['HIT']['HITId'])
    print("https://workersandbox.mturk.com/mturk/preview?groupId=" + new_hit['HIT']['HITGroupId'])
    return new_hit
    
hit_list = []
for filename in sorted(os.listdir(hits_dir))[:50]:
    if '.ipynb' in filename:
        continue
    file = os.path.join(hits_dir, filename)
    
    if not os.path.isdir(file):
        with open(file, 'r') as f:
            question = f.read()
            print(filename)
            hit = define_hit(question)
            print(hit['HIT']['HITId'])
            hit_list.append(hit)
    
# Write a manifest file
hit_list_file = 'hit_list.txt'
with open(hit_list_file, 'w') as f:
    f.write('\n'.join([hit['HIT']['HITId'] for hit in hit_list]))

In [22]:
for hit in hit_list:
    print(hit['HIT']['HITId'])

3IQ9O0AYW5YYYGNSN7ZD2PMU9SVTIB
3NSM4HLQNQTYNYJJJBAR6M2TGVDQQX
3E24UO25QYP2S4NJPP4PBCQ21TK6O9
3DA79LNS58UJ8D5924SQYJK5QAZ3TM
3VLL1PIENPN4GE7URU108AJX6XGOZH
3ULIZ0H1V94LD8G4E4W97AZUCNZ151
37M4O367VIHI9X2T7QF7R2NXAYTM5T
3X2YVV51PT3FQRVOA5JIXAC7WK2W1C
3I4E7AFQ2JYUTR7Y5PKDEQ1UM7KTJN
38LRF35D5KVY8QKY29I03F679O63UD
3M4KL7H8KUMHWCGIJN6EZGDU8IA16H
31SIZS5W58EYZZYJWQZRBCCYCHFQRY
3J5XXLQDHLAYL59Y7ZS5Z18QNVF3V2
3SA4EMRVJU1PZTD5201NRBL1YVTP07
3W9XHF7WGJUP4BDGNAKPF5171WKTKM
3HUR21WDDTOR5XBHWZMDDG1OC7HYX5
3Z56AA6EK3ZES127C9PCJ81X6T4M69
3EHIMLB7F6YF41KKER6XBHPSV2W8H9
3566S7OX5CI00CUN2XB65UDQ3TU17G
3JMNNNO3B03MFCDKT9JW2TMCOQGW2T
33BFF6QPI0AB0U93X9X1L3R4U4H3WC
33BFF6QPI0AB0U93X9X1L3R4U4GW34
31S7M7DAGFP0OHVY8AWQU5EMQWPTLP
3S37Y8CWI7ZWIQS7XN2KV8JA84CW4P
37SDSEDIN8140QZ6C23CJU9LED0183
31ODACBENTEA0EYWBFZW1MHUYJYQSM
3KVQ0UJWPWKEE9W7BSLU00PO807W5U
3TZDZ3Y0JR54EVIGHU9QJQ4WYJI19S
3K2CEDRACA0KUZDJ8MX5UITBQ1FTMR
39TX062QX0NQPUET7ETNNM5B36N3X5
3XJOUITW8TQBFEBB0F4MBRDG02CQTB
3ZCC2DXSD63G521R1V8FWU8XEDBYYB
368IUKXG

OG Experiment

question_0.xml
3IQ9O0AYW5YYYGNSN7ZD2PMU9SVTIB
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_1.xml
3NSM4HLQNQTYNYJJJBAR6M2TGVDQQX
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_10.xml
3E24UO25QYP2S4NJPP4PBCQ21TK6O9
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_100.xml
3DA79LNS58UJ8D5924SQYJK5QAZ3TM
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_101.xml
3VLL1PIENPN4GE7URU108AJX6XGOZH
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_102.xml
3ULIZ0H1V94LD8G4E4W97AZUCNZ151
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_103.xml
37M4O367VIHI9X2T7QF7R2NXAYTM5T
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_104.xml
3X2YVV51PT3FQRVOA5JIXAC7WK2W1C
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_105.xml
3I4E7AFQ2JYUTR7Y5PKDEQ1UM7KTJN
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_106.xml
38LRF35D5KVY8QKY29I03F679O63UD
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_107.xml
3M4KL7H8KUMHWCGIJN6EZGDU8IA16H
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_108.xml
31SIZS5W58EYZZYJWQZRBCCYCHFQRY
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_109.xml
3J5XXLQDHLAYL59Y7ZS5Z18QNVF3V2
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_11.xml
3SA4EMRVJU1PZTD5201NRBL1YVTP07
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_110.xml
3W9XHF7WGJUP4BDGNAKPF5171WKTKM
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_111.xml
3HUR21WDDTOR5XBHWZMDDG1OC7HYX5
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_112.xml
3Z56AA6EK3ZES127C9PCJ81X6T4M69
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_113.xml
3EHIMLB7F6YF41KKER6XBHPSV2W8H9
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_114.xml
3566S7OX5CI00CUN2XB65UDQ3TU17G
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_115.xml
3JMNNNO3B03MFCDKT9JW2TMCOQGW2T
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_116.xml
33BFF6QPI0AB0U93X9X1L3R4U4H3WC
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_117.xml
33BFF6QPI0AB0U93X9X1L3R4U4GW34
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_118.xml
31S7M7DAGFP0OHVY8AWQU5EMQWPTLP
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_119.xml
3S37Y8CWI7ZWIQS7XN2KV8JA84CW4P
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_12.xml
37SDSEDIN8140QZ6C23CJU9LED0183
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_120.xml
31ODACBENTEA0EYWBFZW1MHUYJYQSM
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_121.xml
3KVQ0UJWPWKEE9W7BSLU00PO807W5U
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_122.xml
3TZDZ3Y0JR54EVIGHU9QJQ4WYJI19S
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_123.xml
3K2CEDRACA0KUZDJ8MX5UITBQ1FTMR
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_124.xml
39TX062QX0NQPUET7ETNNM5B36N3X5
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_125.xml
3XJOUITW8TQBFEBB0F4MBRDG02CQTB
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_126.xml
3ZCC2DXSD63G521R1V8FWU8XEDBYYB
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_127.xml
368IUKXGA47KD4ZK4PH40CVSBYG6PQ
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_128.xml
3HXCEECSQLSGASBWXHAYA1HZK7FYZ2
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_129.xml
3SU800BH85RQFRQ00KUWGNZIJGJQU2
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_13.xml
30ZKOOGW2V5IJESL90NQFLFG4101A7
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_130.xml
3U74KRR67LKQCHYJKNC57XIBVRPTNR
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_131.xml
3CMIQF80GMP5DG0Z42WT0HL2GU26QH
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_132.xml
3CESM1J3EH2AXDWLGBVZS63O4VIW6A
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_133.xml
3IHWR4LC7CCF0WV4K9OIBTQ7VFB8I2
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_134.xml
3RBI0I35XD2FPZ43CAFP60CK5CH3YB
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_135.xml
33EEIIWHK66QS6F05A41C911XNSQVR
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_136.xml
3ZZAYRN1I5Q8U2YVL2CIMMIGL1UTO6
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_137.xml
3QMELQS6Y4A5PHFZHHLT57V7CG46RI
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_138.xml
3R5OYNIC2B8QF2AW02PXBMN6V6QTPN
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_139.xml
39WICJI5ASRFUPE88WH8K7LMB6L3Z2
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_14.xml
3I7SHAD35LVXW1GCVJU4PM1T14OM78
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_140.xml
3VGET1QSZZYT1DAQZL0RYK3KZ62W79
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_141.xml
306996CF6VJN2OXDFENMAWZMMJL1B2
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_142.xml
3NKW03WTLL637VF5VK9XYBKF4WUQW1
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_143.xml
3XJOUITW8TQBFEBB0F4MBRDG02CTQE
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_144.xml
3IV1AEQ4DQCBV7FA2R9INU578U08JE
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_145.xml
337F8MIIMYC3A0RM4G9ZODA67ZY40W
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_146.xml
3QI9WAYOGPAHQWFCW6LYVH03YIN6S6
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_147.xml
3X2LT8FDHVHX1RF99QSXCKZFAQ8W8W
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_148.xml
37VUR2VJ69O6CTPJTEJHLG5444L1C9
https://workersandbox.mturk.com/mturk/preview?groupId=3M2FS5OFX87YOG9PGC4X33BP3RTISH
question_149.xml


3IQ9O0AYW5YYYGNSN7ZD2PMU9SVTIB
3NSM4HLQNQTYNYJJJBAR6M2TGVDQQX
3E24UO25QYP2S4NJPP4PBCQ21TK6O9
3DA79LNS58UJ8D5924SQYJK5QAZ3TM
3VLL1PIENPN4GE7URU108AJX6XGOZH
3ULIZ0H1V94LD8G4E4W97AZUCNZ151
37M4O367VIHI9X2T7QF7R2NXAYTM5T
3X2YVV51PT3FQRVOA5JIXAC7WK2W1C
3I4E7AFQ2JYUTR7Y5PKDEQ1UM7KTJN
38LRF35D5KVY8QKY29I03F679O63UD
3M4KL7H8KUMHWCGIJN6EZGDU8IA16H
31SIZS5W58EYZZYJWQZRBCCYCHFQRY
3J5XXLQDHLAYL59Y7ZS5Z18QNVF3V2
3SA4EMRVJU1PZTD5201NRBL1YVTP07
3W9XHF7WGJUP4BDGNAKPF5171WKTKM
3HUR21WDDTOR5XBHWZMDDG1OC7HYX5
3Z56AA6EK3ZES127C9PCJ81X6T4M69
3EHIMLB7F6YF41KKER6XBHPSV2W8H9
3566S7OX5CI00CUN2XB65UDQ3TU17G
3JMNNNO3B03MFCDKT9JW2TMCOQGW2T
33BFF6QPI0AB0U93X9X1L3R4U4H3WC
33BFF6QPI0AB0U93X9X1L3R4U4GW34
31S7M7DAGFP0OHVY8AWQU5EMQWPTLP
3S37Y8CWI7ZWIQS7XN2KV8JA84CW4P
37SDSEDIN8140QZ6C23CJU9LED0183
31ODACBENTEA0EYWBFZW1MHUYJYQSM
3KVQ0UJWPWKEE9W7BSLU00PO807W5U
3TZDZ3Y0JR54EVIGHU9QJQ4WYJI19S
3K2CEDRACA0KUZDJ8MX5UITBQ1FTMR
39TX062QX0NQPUET7ETNNM5B36N3X5
3XJOUITW8TQBFEBB0F4MBRDG02CQTB
3ZCC2DXSD63G521R1V8FWU8XEDBYYB
368IUKXGA47KD4ZK4PH40CVSBYG6PQ
3HXCEECSQLSGASBWXHAYA1HZK7FYZ2
3SU800BH85RQFRQ00KUWGNZIJGJQU2
30ZKOOGW2V5IJESL90NQFLFG4101A7
3U74KRR67LKQCHYJKNC57XIBVRPTNR
3CMIQF80GMP5DG0Z42WT0HL2GU26QH
3CESM1J3EH2AXDWLGBVZS63O4VIW6A
3IHWR4LC7CCF0WV4K9OIBTQ7VFB8I2
3RBI0I35XD2FPZ43CAFP60CK5CH3YB
33EEIIWHK66QS6F05A41C911XNSQVR
3ZZAYRN1I5Q8U2YVL2CIMMIGL1UTO6
3QMELQS6Y4A5PHFZHHLT57V7CG46RI
3R5OYNIC2B8QF2AW02PXBMN6V6QTPN
39WICJI5ASRFUPE88WH8K7LMB6L3Z2
3I7SHAD35LVXW1GCVJU4PM1T14OM78
3VGET1QSZZYT1DAQZL0RYK3KZ62W79
306996CF6VJN2OXDFENMAWZMMJL1B2
3NKW03WTLL637VF5VK9XYBKF4WUQW1
3XJOUITW8TQBFEBB0F4MBRDG02CTQE
3IV1AEQ4DQCBV7FA2R9INU578U08JE
337F8MIIMYC3A0RM4G9ZODA67ZY40W
3QI9WAYOGPAHQWFCW6LYVH03YIN6S6
3X2LT8FDHVHX1RF99QSXCKZFAQ8W8W
37VUR2VJ69O6CTPJTEJHLG5444L1C9

# Interpret results

In [5]:
import xmltodict
import boto3
MTURK_SANDBOX = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'
mturk = boto3.client('mturk',
   aws_access_key_id = "secret",
   aws_secret_access_key = "secret",
   region_name='us-east-1',
#    endpoint_url = MTURK_SANDBOX
)

hit_id = '3EHIMLB7F6YF41KKER6XBHPSV2W8H9'
worker_results = mturk.list_assignments_for_hit(HITId=hit_id)
# worker_results['Answer']

In [ ]:
unk_tally = 0
no_unk = []
for debate in debates():
    if '<unk' in debate:
        unk_tally += 1
    else:
        no_unk.append(debate)
        
print('unk', unk_tally)
print('no unk')

for d in no_unk:
    print(d)

### Retrieve HIT results from mTurk

In [6]:
from collections import defaultdict

# hit_id = '3E24UO25QYP2S4NJPP4PBCQ21TK6O9'

def get_responses(hit_id, responses):
    
    worker_results = mturk.list_assignments_for_hit(HITId=hit_id)
    for assignment in worker_results['Assignments']:
        xml = xmltodict.parse(assignment['Answer'])
        questions = xml['QuestionFormAnswers']['Answer']
        for q in questions:
            category = q['QuestionIdentifier']
            response = q['FreeText']

            if responses[hit_id].get(category) is None:
                responses[hit_id][category] = [response]
            else: 
                responses[hit_id][category].append(response)

    return responses

og_responses = defaultdict(dict)
for hit_id in og_hit_ids:
    og_responses = get_responses(hit_id, og_responses)
    
# Read list of human hit ids
human_hit_ids = open('human_hit_list.txt', 'r').readlines()
human_hit_ids = [hit.strip() for hit in human_hit_ids]
    
human_responses = defaultdict(dict)

for hit_id in human_hit_ids:
    print(hit_id)
    human_responses = get_responses(hit_id, human_responses)


3IZPORCT1E8LM6ZGD0CX2YKXLWEHRS
3JYPJ2TAYH7GWXSV5IK8UI854ZDFP0
34O39PNDK57BVTWUD821GNHZQJ9RBM
34XASH8KLPLM27KCR5OLP8SE10PMPE
3K8CQCU3KD0I4Y2DFSHGUHPAZN2WNJ
3OWZNK3RYKO9N8OFY94VK51F3C0U2R
3VI0PC2ZAXJFBJ7FQCDFBP3MYZDOXJ
374UMBUHN4O7LD1SVX3RRMOZWP6CTR
3P0I4CQYVX60MJ2PG7HT96PFPX7WOY
3IVKZBIBJZ8XNLZTSPC2S8PT7YXHSG
32FESTC2NGPMLQGHV2T1WIA1F3DCUI
38LRF35D5KVY8QKY29I03F679Q0U32
3BDORL6HKJCU5YO0R8YWR7NH849RCT
3DZKABX2ZH4MY55H0S36S4CKTAMCV7
3DFYDSXB2VZ9T4MN5UANJMNW5NMJU3
3H6W48L9F3OI7JEQV7U8Y6U5Z23WPF
3EQVJH0T4ZKY2LC8HPHS2DLF9HBHT5
3PCPFX4U4ZP1W9TA5VZXUNYF9VZFQR
3XDJY5RK5RKJQM322NNJDKYDNQWU4N
3D0LPO3EAAY4BOXPV8ZHU3AV057OYP
30Y6N4AHYOV4AQUER4T7BD5ZT4LRDT
3X52SWXE0W4ZDU5MQ282E6VY0JOCWH
3NKW03WTLL637VF5VK9XYBKF4YPWQ6
33CLA8O0MHA18A8AIAOXZD8K5H1FRS
3ABAOCJ4R7372JLRRI3APDIO6WBMQ5
3P520RYKCG51M572GS6TIC4RNMRU5S
3VLL1PIENPN4GE7URU108AJX6ZBOZG
3LG268AV37592W8ENZ4RHVNKTGMREM
33NKDW9FFWHE2UAC074OGP959LUCXA
31KSVEGZ33R3JWU58ZYX31UK0KRWR7
391FPZIE4BLD2YRXHU72797HSVIHUW
338GLSUI42AD9P8NXZO2PNDGRJKFSG
3OB6JN3A

In [9]:
print(len(human_responses), len(og_responses))

50 56


### Crunch the stats

In [14]:
import statistics
from collections import defaultdict

def stats(responses):
    
    ratings = defaultdict(list)
    
    for i, hit_tallies in responses.items():
        for key, rating_list in hit_tallies.items():
            ratings[key] += [int(rating) for rating in rating_list]
            
    # Compute stats
    for key, values in ratings.items():
        print(key)
        print('Mean: {} median: {} mode: {} stddev {}'.format(
            statistics.mean(values),
            statistics.median(values),
            statistics.mode(values),
            statistics.stdev(values)
        ))
        
    # Compute total responses
    length = 0
    
    for key, values in ratings.items():
        length += len(values)
        print(key, len(values))
    print(length, 'responses')
    

print('Human Responses')
stats(human_responses)

print('\nGen Responses')
stats(og_responses)

Human Responses
style
Mean: 3.2730923694779115 median: 3 mode: 4 stddev 0.7224223333506505
content
Mean: 3.392 median: 3.0 mode: 3 stddev 0.626022055887675
strategy
Mean: 3.412 median: 3.0 mode: 3 stddev 0.5963880572743201
quality
Mean: 3.512 median: 4.0 mode: 4 stddev 0.5960512632599103
style 249
content 250
strategy 250
quality 250
999 responses

Gen Responses
style
Mean: 3.1535714285714285 median: 3.0 mode: 3 stddev 0.8683945318690294
content
Mean: 3.289285714285714 median: 3.0 mode: 4 stddev 0.770346142797835
strategy
Mean: 3.1335740072202167 median: 3 mode: 3 stddev 0.8210495265932677
quality
Mean: 3.192857142857143 median: 3.0 mode: 4 stddev 0.8243292359222185
style 280
content 280
strategy 277
quality 280
1117 responses


In [3]:
og_hit_ids = [
    '3IQ9O0AYW5YYYGNSN7ZD2PMU9SVTIB',
    '3NSM4HLQNQTYNYJJJBAR6M2TGVDQQX',
    '3E24UO25QYP2S4NJPP4PBCQ21TK6O9',
    '3DA79LNS58UJ8D5924SQYJK5QAZ3TM',
    '3VLL1PIENPN4GE7URU108AJX6XGOZH',
    '3ULIZ0H1V94LD8G4E4W97AZUCNZ151',
    '37M4O367VIHI9X2T7QF7R2NXAYTM5T',
    '3X2YVV51PT3FQRVOA5JIXAC7WK2W1C',
    '3I4E7AFQ2JYUTR7Y5PKDEQ1UM7KTJN',
    '38LRF35D5KVY8QKY29I03F679O63UD',
    '3M4KL7H8KUMHWCGIJN6EZGDU8IA16H',
    '31SIZS5W58EYZZYJWQZRBCCYCHFQRY',
    '3J5XXLQDHLAYL59Y7ZS5Z18QNVF3V2',
    '3SA4EMRVJU1PZTD5201NRBL1YVTP07',
    '3W9XHF7WGJUP4BDGNAKPF5171WKTKM',
    '3HUR21WDDTOR5XBHWZMDDG1OC7HYX5',
    '3Z56AA6EK3ZES127C9PCJ81X6T4M69',
    '3EHIMLB7F6YF41KKER6XBHPSV2W8H9',
    '3566S7OX5CI00CUN2XB65UDQ3TU17G',
    '3JMNNNO3B03MFCDKT9JW2TMCOQGW2T',
    '33BFF6QPI0AB0U93X9X1L3R4U4H3WC',
    '33BFF6QPI0AB0U93X9X1L3R4U4GW34',
    '31S7M7DAGFP0OHVY8AWQU5EMQWPTLP',
    '3S37Y8CWI7ZWIQS7XN2KV8JA84CW4P',
    '37SDSEDIN8140QZ6C23CJU9LED0183',
    '31ODACBENTEA0EYWBFZW1MHUYJYQSM',
    '3KVQ0UJWPWKEE9W7BSLU00PO807W5U',
    '3TZDZ3Y0JR54EVIGHU9QJQ4WYJI19S',
    '3K2CEDRACA0KUZDJ8MX5UITBQ1FTMR',
    '39TX062QX0NQPUET7ETNNM5B36N3X5',
    '3XJOUITW8TQBFEBB0F4MBRDG02CQTB',
    '3ZCC2DXSD63G521R1V8FWU8XEDBYYB',
    '368IUKXGA47KD4ZK4PH40CVSBYG6PQ',
    '3HXCEECSQLSGASBWXHAYA1HZK7FYZ2',
    '3SU800BH85RQFRQ00KUWGNZIJGJQU2',
    '30ZKOOGW2V5IJESL90NQFLFG4101A7',
    '3U74KRR67LKQCHYJKNC57XIBVRPTNR',
    '3CMIQF80GMP5DG0Z42WT0HL2GU26QH',
    '3CESM1J3EH2AXDWLGBVZS63O4VIW6A',
    '3IHWR4LC7CCF0WV4K9OIBTQ7VFB8I2',
    '3RBI0I35XD2FPZ43CAFP60CK5CH3YB',
    '33EEIIWHK66QS6F05A41C911XNSQVR',
    '3ZZAYRN1I5Q8U2YVL2CIMMIGL1UTO6',
    '3QMELQS6Y4A5PHFZHHLT57V7CG46RI',
    '3R5OYNIC2B8QF2AW02PXBMN6V6QTPN',
    '39WICJI5ASRFUPE88WH8K7LMB6L3Z2',
    '3I7SHAD35LVXW1GCVJU4PM1T14OM78',
    '3VGET1QSZZYT1DAQZL0RYK3KZ62W79',
    '306996CF6VJN2OXDFENMAWZMMJL1B2',
    '3NKW03WTLL637VF5VK9XYBKF4WUQW1',
    '3XJOUITW8TQBFEBB0F4MBRDG02CTQE',
    '3IV1AEQ4DQCBV7FA2R9INU578U08JE',
    '337F8MIIMYC3A0RM4G9ZODA67ZY40W',
    '3QI9WAYOGPAHQWFCW6LYVH03YIN6S6',
    '3X2LT8FDHVHX1RF99QSXCKZFAQ8W8W',
    '37VUR2VJ69O6CTPJTEJHLG5444L1C9'
]

In [30]:
print(len(og_hit_ids))

56
